In [ ]:
from gensim.models import Word2Vec as wtv
from nltk import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/spam.csv',encoding= "ISO-8859-1")

In [ ]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
df = df.drop(['Unnamed: 2',	'Unnamed: 3',	'Unnamed: 4'], axis =1)

In [ ]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
df['v2'][0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [ ]:
df.v1.value_counts()

ham     4825
spam     747
Name: v1, dtype: int64

In [ ]:
#importing simple preprocess
from gensim.utils import simple_preprocess
processed = df.v2.apply(lambda x: simple_preprocess(x))

In [ ]:
processed[0]

['go',
 'until',
 'jurong',
 'point',
 'crazy',
 'available',
 'only',
 'in',
 'bugis',
 'great',
 'world',
 'la',
 'buffet',
 'cine',
 'there',
 'got',
 'amore',
 'wat']

In [ ]:
processed.head()

0    [go, until, jurong, point, crazy, available, o...
1                          [ok, lar, joking, wif, oni]
2    [free, entry, in, wkly, comp, to, win, fa, cup...
3       [dun, say, so, early, hor, already, then, say]
4    [nah, don, think, he, goes, to, usf, he, lives...
Name: v2, dtype: object

In [ ]:
#import word2vec
from gensim.models import Word2Vec
# create Cbow and skip gram
# cbow model
# window- dist between the target and surrounding words default 5
# min count of words taken
# sg ==1 for skip gram, for cbow  sg=0
cbow_w2v_model = Word2Vec(processed, size =300, window =9, min_count =2, sg=0 )
#skipgram model
#train a word2vec model from the given dataset
skgram_w2v_model = Word2Vec(processed, size = 300, window =9, min_count =2, sg=1)
# vocabulary size
print('skgram model vocabulary size: ', len(skgram_w2v_model.wv.vocab))

skgram model vocabulary size:  3892


In [ ]:
#cbow model vocabulary
list(cbow_w2v_model.wv.vocab.items())[:20]

[('go', <gensim.models.keyedvectors.Vocab at 0x7fd3be2a2750>),
 ('until', <gensim.models.keyedvectors.Vocab at 0x7fd3be124a50>),
 ('point', <gensim.models.keyedvectors.Vocab at 0x7fd3be124350>),
 ('crazy', <gensim.models.keyedvectors.Vocab at 0x7fd3be124610>),
 ('available', <gensim.models.keyedvectors.Vocab at 0x7fd3be11f8d0>),
 ('only', <gensim.models.keyedvectors.Vocab at 0x7fd3be11f810>),
 ('in', <gensim.models.keyedvectors.Vocab at 0x7fd3be11fd50>),
 ('bugis', <gensim.models.keyedvectors.Vocab at 0x7fd3be11fc90>),
 ('great', <gensim.models.keyedvectors.Vocab at 0x7fd3be0f0c50>),
 ('world', <gensim.models.keyedvectors.Vocab at 0x7fd3be11f210>),
 ('la', <gensim.models.keyedvectors.Vocab at 0x7fd3be11ff90>),
 ('buffet', <gensim.models.keyedvectors.Vocab at 0x7fd3be11f690>),
 ('cine', <gensim.models.keyedvectors.Vocab at 0x7fd3be11f1d0>),
 ('there', <gensim.models.keyedvectors.Vocab at 0x7fd3be11fe10>),
 ('got', <gensim.models.keyedvectors.Vocab at 0x7fd3be11fb50>),
 ('wat', <gensim.m

In [ ]:
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
model = Word2Vec(processed)
word_vectors = model.wv
word_vectors.save('vectors.kv')
google_w2vec = KeyedVectors.load('vectors.kv')

#load into gensim pretrained model
#google_w2vec = KeyedVectors.load_word2vec_format(processed,binary =True)

In [ ]:
model.wv

In [ ]:

def get_embedding_w2v(doc_tokens, pre_trained):
  embeddings = []
  if pre_trained:
    model = google_w2vec
  else:
    model = cbow_w2v_model
  # iterate over tokens  to extract their vectors
  for tok in doc_tokens:
    if tok in model.wv.vocab:
      embeddings.append(model.wv.word_vec(tok))  
    # take mean of individual words to get the vector of statement
    return np.mean(embeddings, axis =0)  

In [ ]:
# general preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
# create X from w2vec
X_w2v_model = processed.apply(lambda x :get_embedding_w2v(x,pre_trained =0) ) #0---our model
#X_w2v_model = pd.DataFrame(list(X_w2v_model))
pd.DataFrame(X_w2v_model.to_list())
print('X shape:', X_w2v_model.shape)

TypeError: ignored